### In this notebook, we'll compile a bit of additional reference data from outside sources and add them to our cleaned dataset to enrich the model.

We use the Yahoo Finance Python API and data from FRED Economic Data for our additional variables.

Our original dataset taken from Kaggle already has numerous features, but let's add just a few more to enrich our model. We'll be adding the following information to our dataset:

**1. Risk-free rate** - The risk-free rate, which is the theoretical rate of return on an investment with zero risk, impact's a company's considerations in a few ways. For a company's stock or bond to be an attractive investment, the company must offer a certain premium above the risk-free rate, which can impact the company's agenda or goals, atleast for certain time periods, depending on the risk-free rate. The risk-free rate is also an input in the cost of equity formula, which also influences a company's weighted average cost of capital. Such things would have an influence on the company's balance sheet, thus, we include this variable in our dataset. A common proxy for the risk-free rate is the yield on 3-month treasury bills. While no investment is truly risk-free, 3-mo treasury bills are very close. For a backup source for risk-free rate in case the 3-mo treasury bill yield isn't available for a particular date, we will use the fed-funds rate.

**2. 10-yr Treasury Note Yield** - Systematic risk plays a part in the performance of any company, so it's important to consider certain macro-economic indicators to gauge the state of the economy a company is placed in. One such indicator is the current interest-rate environment, to which we use the 10-year Treasury Note yield as a proxy to gauge this. This yield is said to be an indicator of broader investor confidence in the market. When yields are low and prices are high, investor confidence in the economy is not so high because the broad market is shifting towards safer investments, and when yields are high and prices are low, it signals low demand for treasury notes and higher demand elsewhere, meaning investors have an appetite for and the comfort of taking on riskier investments with possible higher rates of returns.

**3. Stock Price** - While a company's current stock price alone at any point in time does not give much info, other info can be derived from the stock price. For one, we can tell whether a company publicly listed or not based on whether the price is available on data platforms that pull prices from exchanges (In our case, we are using the YahooFinance API, so we assume that if there is no response to our price request from yfinance, then the stock is not publicly listed. How bold of an assumption this is is still ambiguous, as it is a free API after all and may not have the same depth of available data as a paid data provider). The other thing we can do using stock prices is take a look at the movement of the price using a technical analysis perspective to gauge market sentiment on the company.

**4. YTD Equity Return** - Another derivation made from stock prices, we can also take into consideration the company's YTD equity return, if available, to analyze the company's current or prospective performance during the time period of interest. As both credit risk and stock prices are forward-looking indicators, looking at YTD price changes may help us, to some degree, with assumptions of credit risk as well. An increasing price means that market sentiment on the future outlook, namely financials, of the company have improved. If the market is correct, then this improved financial stability would also result in improved capabilities to pay obligations, hence reduced credit risk.

**5. Last Dividend Amount and Date** - A dividend payout is a very big decision for any company, and it signifies a few things. Namely, a dividend payout symbolizes financial strength, as a company is confident enough of its free cash flow projections for the future to decide to periodically pay out some cash to its shareholders. Dividend's are generally paid by mature companies who have solidified their position in the market and are not in any growth stage. On the other hand, companyies in a growth stage may reinvest all their earnings back into the company for future growth. In this case, investors do not receive dividend payouts but get compensated via capital gains depending on the actual growth prospects of the company. 

**6. Consumer Price Index (CPI)** - The CPI is an indicator for inflation. Inflation impacts all aspects of a sovereignty's economy. During periods of increasing inflation, companies seem to be doing well since their revenues seem to be increasing, as well as earnings. However, this is misleading as the real value of money is decreasing. This is an important consideration to make when doing analysis on data points that are far apart in time period. For exmaple, if we are comparing earnings of a company in 2015, and earnings of another company, or even the same company, in 2005, we have to take inflation into consideration to gauge the real increase or decrease of earnings. When investors in the broad market are worried about inflation, this may seep into a company's operations, as they have to now offer higher rates on their fixed income issues, which impacts the company's cost of capital and domino effects into many other aspects of the company's operations.


In [1]:
import pandas as pd
import yfinance as yf

In [2]:
df = pd.read_csv('working_datasets/df_cleaned.csv')
three_mo_tb_yields_df = pd.read_csv('input_datasets/3_mo_treasury_yields.csv')
ten_yr_tn_yields_df = pd.read_csv('input_datasets/10_yr_treasury_yields.csv')
fed_funds_rate_df = pd.read_csv('input_datasets/daily_fed_funds_rate.csv')
cpi_index_df = pd.read_csv("input_datasets/us_cpi_index.csv")
us_corp_bond_index_ig_yield_df = pd.read_csv("input_datasets/baml_us_corp_bond_ig_yield_index.csv")
us_corp_bond_index_non_ig_yield_df = pd.read_csv("input_datasets/baml_us_corp_bond_non-ig_yield_index.csv")

In [3]:
#convert date columns in every dataframe to pandas datetime.date format
for df_dummy in [
    df, 
    three_mo_tb_yields_df, 
    ten_yr_tn_yields_df,
    fed_funds_rate_df, 
    cpi_index_df, 
    us_corp_bond_index_ig_yield_df,
    us_corp_bond_index_non_ig_yield_df
]:
    for col in df_dummy.columns:
        if col.lower() == 'date':
            df_dummy[col] = pd.to_datetime(df_dummy[col]).dt.date

In [4]:
def get_risk_free_rate(date):
   
    """
    Retreives risk-free rate for date passed in.
    Primarily uses the 3-mo treasury yield, but
    if that is not available, then the fed funds rate
    """
    try:
        rfr = float(three_mo_tb_yields_df[three_mo_tb_yields_df['DATE'] == date]['DTB3'].values[0])
    except:
        rfr = float(fed_funds_rate_df[fed_funds_rate_df['DATE'] == date]['DFF'].values[0])
    return rfr

In [5]:
def get_t_note_yield(date):
    """
    Retrevies the 10 yr T-Note yield for date passed in
    """
    try:
        return float(ten_yr_tn_yields_df[ten_yr_tn_yields_df['DATE'] == date]['DGS10'].values[0])
    except:
        return None

In [6]:
def get_stock_price(date, ticker):
    
    """
    Retreives the closing stock price for data passed in
    """
    start_date = date
    end_date = date + pd.DateOffset(1)
    try:
        price = yf.Ticker(ticker).history(
            start=start_date, 
            end=end_date
        )['Close'].values[0]
    except IndexError:
        price = None
    return price

In [7]:
def get_ytd_stock_return(date, ticker, current_price):
    
    """
    Calculates YTD return for specified ticker
    """
    start_date = date - pd.tseries.offsets.YearBegin() + pd.DateOffset(1)
    end_date = start_date + pd.DateOffset(1)
    start_price = None
    iterations = 0
    while not start_price and iterations <=7:
        try:
            start_price = yf.Ticker(ticker).history(
                start=start_date, 
                end=end_date
            )['Close'].values[0]
            iterations = 0
        except IndexError:
            start_date += pd.DateOffset(1)
            end_date += pd.DateOffset(1)
            iterations += 1
    if iterations <= 7:
        return (current_price-start_price)/start_price
    else:
        return None

In [8]:
def get_last_dividend_info(date, ticker):
    """
    Retrieves the amount and date of the last dividend payment.
    Searches for dividends up to 365 days before date passed in.
    If no dividend found then, we assume that the company did not issue dividends
    """
    dividend_df = yf.Ticker(ticker).dividends
    if not dividend_df.empty:
        dividend_df.index = [pd.to_datetime(elem).date() for elem in dividend_df.index]
        last_div_date = date
        iterations = 0
        while last_div_date not in dividend_df.index and iterations <= 365:
            last_div_date = (last_div_date - pd.DateOffset(1)).date()
            iterations+=1
        if iterations <= 365:
            last_div_amount = dividend_df[last_div_date]
            return last_div_amount, last_div_date
        else:
            return None, None
    else:
        return None, None

In [9]:
def get_cpi_index(date):
    
    """
    Retreive CPI index level for month of date passed in
    """
    cpi_date = (pd.to_datetime(date) - pd.offsets.MonthBegin()).date()
    return cpi_index_df[cpi_index_df['DATE'] == cpi_date]['CPIAUCSL'].values[0]

In [10]:
#the yfinance methods used are highly verbose, and this cell 
#execution will output a large amount of print statements for 
#failed data requests (which are expected), so we'll supress the 
#output of print function altogether since there was no other
#simple solution found online for disabling yfinance print outputs
import sys, os
sys.stdout = open(os.devnull, 'w')

df['risk_free_rate'] = df['Date'].map(lambda date: get_risk_free_rate(date))
df['ten_yr_t_note_yield'] = df['Date'].map(lambda date: get_t_note_yield(date))
df['stock_price'] = df.apply(lambda row: get_stock_price(row['Date'], row['Symbol']), axis=1)
df['ytd_equity_return'] = df.apply(lambda row: get_ytd_stock_return(row['Date'], row['Symbol'], row['stock_price']), axis=1)
df['last_div_amount'] = df.apply(lambda row: get_last_dividend_info(row['Date'], row['Symbol'])[0], axis=1)
df['last_div_date'] = df.apply(lambda row: get_last_dividend_info(row['Date'], row['Symbol'])[1], axis=1)
df['cpi_index'] = df['Date'].map(lambda date: get_cpi_index(date))
df.fillna(0, inplace=True)

In [11]:
#export compiled dataset 
df.to_csv('working_datasets/df_cleaned_compiled.csv', index=False)

In [22]:
#update metadata
df_metadata = pd.read_csv('working_datasets/df_cleaned_metadata.csv')
cols_to_add = [
    'risk_free_rate', 
    'ten_yr_t_note_yield', 
    'stock_price', 
    'ytd_equity_return', 
    'last_div_amount', 
    'last_div_date',
    'cpi_index'
]
df_metadata = df_metadata.append(pd.DataFrame(cols_to_add, columns=['continuous_cols']))
#export metadata
df_metadata.to_csv('working_datasets/df_cleaned_compiled_metadata.csv', index=False)